In [82]:
from azureml.core import Workspace, Dataset, Datastore
import os
from azureml.core.authentication import InteractiveLoginAuthentication
from azureml.core import Run

import azureml.core
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core import ComputeTarget

from azureml.pipeline.steps import PythonScriptStep
from azureml.core import Experiment, Environment
from azureml.core.runconfig import RunConfiguration
from azureml.pipeline.core import Pipeline



In [83]:
pip install azureml-pipeline

Note: you may need to restart the kernel to use updated packages.


In [84]:
ws = Workspace.from_config("Users/mypersonall3099/Online_retail/config.json") 


In [85]:
compute_name = "OnlineRetail"
#vm_size = "Standard_E4ds_v4"
vm_size = "Standard_E8ds_v4"
compute_target = ws.compute_targets[compute_name]

In [86]:
from azureml.core import Environment

env = Environment.from_conda_specification(name="myenv", file_path="environment.yaml")
env.register(workspace=ws)



{
    "assetId": "azureml://locations/eastus2/workspaces/7ce76f32-f5fe-4dd3-b900-96fb92b58ebb/environments/myenv/versions/1",
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:20240709.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "buildContext": null,
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "myenv",
    "python": {
       

In [91]:
from azureml.core import Environment

env = Environment.get(workspace=ws, name="myenv")
print(env.python.conda_dependencies.serialize_to_string())


name: myenv
channels:
- defaults
dependencies:
- python=3.7
- pandas
- scikit-learn
- statsmodels
- pip
- pip:
  - pmdarima
  - azureml-sdk
  - azureml-dataset-runtime[fuse, pandas]
  - seaborn
  - joblib



In [97]:
#Declaring environment
aml_config = RunConfiguration()
aml_config.target = compute_target 

env = Environment.from_conda_specification(name="myenv", file_path="environment.yaml")

In [100]:
print(env.environment_variables)

{'EXAMPLE_ENV_VAR': 'EXAMPLE_VALUE'}


In [101]:
try:
    import sklearn
    import joblib
    print("scikit-learn version:", sklearn.__version__)
    print("joblib version:", joblib.__version__)
except ImportError as e:
    print(f"Import error: {e}")


scikit-learn version: 1.5.0
joblib version: 1.4.2


In [98]:
#Pipeline
read_data = 'data_wrangling.py'
prep = 'preprocessing.py'
model = 'modelling.py'

#Script initialization
py_script_run_read = PythonScriptStep(
                script_name = read_data,
                compute_target = compute_target,
                arguments=['--input-data','online_retail_II.csv'],
                runconfig = aml_config,
                allow_reuse=False)

py_script_run_prep = PythonScriptStep(
                script_name = prep,
                compute_target=compute_target,
                arguments=['--prep','wranggled.csv'],
                runconfig = aml_config,
                allow_reuse=False)

py_script_run_model = PythonScriptStep(
                script_name = model,
                compute_target=compute_target,
                arguments=['--train','preprocessed.csv'],
                runconfig = aml_config,
                allow_reuse=False)

pipeline_steps = [py_script_run_read, py_script_run_prep, py_script_run_model]
pipeline_1 = Pipeline(workspace=ws, steps = pipeline_steps)

In [99]:
# Experiment

pipeline_run = Experiment(ws, "First_run_7").submit(pipeline_1)
pipeline_run.wait_for_completion(show_output=True)

Created step data_wrangling.py [41a13b10][fc8f87cb-8126-4ab9-bd60-eea132e0abe4], (This step will run and generate new outputs)
Created step preprocessing.py [b97d415e][dca2f692-d94b-4a31-a2fa-944608256583], (This step will run and generate new outputs)
Created step modelling.py [e56a30e0][7c6dd2ed-9ebc-41b8-a640-00ac59d67c25], (This step will run and generate new outputs)
Submitted PipelineRun 28116866-3e11-4166-a1a3-6926f2395704
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/28116866-3e11-4166-a1a3-6926f2395704?wsid=/subscriptions/d8c4fe8f-f720-48ac-86e8-880576d9cd7a/resourcegroups/OnlineRetail/workspaces/OnlineRetail&tid=f56f1f69-458e-427b-bada-4cba658f7917
PipelineRunId: 28116866-3e11-4166-a1a3-6926f2395704
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/28116866-3e11-4166-a1a3-6926f2395704?wsid=/subscriptions/d8c4fe8f-f720-48ac-86e8-880576d9cd7a/resourcegroups/OnlineRetail/workspaces/OnlineRetail&tid=f56f1f69-458e-427b-bada-4cba658f7917
PipelineRu

ActivityFailedException: ActivityFailedException:
	Message: Activity Failed:
{
    "error": {
        "code": "UserError",
        "message": "Execution failed. User process '/azureml-envs/azureml_f3f7e6c5fb83d94df23933000bf02da3/bin/python' exited with status code 1. Please check log file 'user_logs/std_log.txt' for error details. Error: Traceback (most recent call last):\n  File \"modelling.py\", line 5, in <module>\n    import joblib\nModuleNotFoundError: No module named 'joblib'\n\n",
        "messageParameters": {},
        "details": []
    },
    "time": "0001-01-01T00:00:00.000Z",
    "componentName": "CommonRuntime"
}
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Activity Failed:\n{\n    \"error\": {\n        \"code\": \"UserError\",\n        \"message\": \"Execution failed. User process '/azureml-envs/azureml_f3f7e6c5fb83d94df23933000bf02da3/bin/python' exited with status code 1. Please check log file 'user_logs/std_log.txt' for error details. Erro

In [ ]:
# Experiment 2

pipeline_run = Experiment(ws, "Second_run").submit(pipeline_1)
pipeline_run.wait_for_completion(show_output=True)

In [1]:
from azureml.core import Workspace, Datastore

# Connect to the workspace
ws = Workspace.from_config()

# Get the datastore
datastore = Datastore.get(ws, 'workspaceblobstore')

# Print details
print("Datastore:", datastore.name)
print("Blob Container Name:", datastore.container_name)
print("Storage Account Name:", datastore.account_name)


Datastore: workspaceblobstore
Blob Container Name: azureml-blobstore-7ce76f32-f5fe-4dd3-b900-96fb92b58ebb
Storage Account Name: onlineretail3582987100
